In [1]:

import scipy.ndimage
%load_ext autoreload
%autoreload 2

import napari
import pyclesperanto as cle
from magicgui import magicgui
from napari.types import ImageData, LabelsData
import matplotlib.pyplot as plt
from mt.utils import load_scan, contact_area
import skimage
import scipy

cle.select_device("RTX")
import cv2

In [1]:
def _segment_particles(scan: ImageData,
                       n_erosions: int = 0,
                       sigma: float = 0.1,
                       radius: int = 1) -> LabelsData:
    mask = cle.eroded_otsu_labeling(input_image=scan,
                                    number_of_erosions=n_erosions,
                                    outline_sigma=sigma)
    # mask = cle.dilate_labels(input_image=mask,
    #                          radius=radius)
    cle.mask(input_image=mask, output_image=mask, mask=mask)
    return mask

from skimage.filters import threshold_multiotsu
import numpy as np


@magicgui(auto_call=True)
def segment_particles(scan: ImageData,
                      n_erosions: int = 2,
                      sigma: float = 0.3,
                      radius: int = 1) -> LabelsData:
    mask = _segment_particles(scan, n_erosions, sigma, radius)
    return mask


# @magicgui(auto_call=True)
def otsu(scan: ImageData,
         otsu_sigma: float = 2.2,
         particle_enlarge_radius: int = 1,
         particle_mask_sigma: float = 0.1,
         particle_erosions: int = 0,
         smooth_labels_radius: int = 2) -> LabelsData:
    
    # create an empty mask array
    mask = np.zeros_like(scan)
    print("Starting particle segmentation")
    precise_particle_mask = cle.pull(_segment_particles(scan,
                                               n_erosions=particle_erosions,
                                               sigma=particle_mask_sigma,
                                               radius=particle_enlarge_radius))

    # apply a gaussian blur to the scan and put into RAM
    im = cle.gaussian_blur(scan, sigma_x=otsu_sigma, sigma_y=otsu_sigma, sigma_z=otsu_sigma)
    im = cle.pull(im)
    print("Starting multiotsu thresholding")
    # apply skimage's multiotsu thresholding
    thresholds = threshold_multiotsu(im, classes=3)
    mask = np.where(im <= thresholds[0], 1, mask)
    mask = np.where((im > thresholds[0]) & (im < thresholds[1]), 2, mask)
    mask = np.where(im >= thresholds[1], 3, mask)
    # apply the more precise particle segmentation over the threshold mask
    mask = cle.np.where(precise_particle_mask,
                        3,
                        mask)
    del im
    mask = cle.smooth_labels(mask, radius=smooth_labels_radius)
    return mask

def reslice(scan: np.ndarray,
            axis: tuple[int, int, int] = (0, 1, 2)) -> np.ndarray:
    return np.transpose(scan, axis)
            

NameError: name 'ImageData' is not defined

In [3]:
path_scan = "../../04_uCT/AD50/"
scan = load_scan(path_scan, logging=True)

Loading images from:  ../../04_uCT/AD50/Slices/
Loaded stack with the shape (1780, 545, 2249) and a size of 4.36 GB in 2.36 s.


In [4]:
scan = reslice(scan[:1000], (1, 0, 2))
scan = scipy.ndimage.gaussian_filter(scan, sigma=2.2)

In [5]:
scan = reslice(scan[:1000], (1, 0, 2))
scan_gpu = cle.push(scan)
scan_gpu = cle.gaussian_blur(scan_gpu, sigma_x=2.2, sigma_y=2.2, sigma_z=2.2)
scan = cle.pull(scan_gpu).astype(np.uint16)
del scan_gpu

In [ ]:
im = cle.push(scan)
min_intensity = cle.minimum_of_all_pixels(im)
max_intensity = cle.maximum_of_all_pixels(im)
hist = cle.histogram(im, min=min_intensity, max=max_intensity, nbins=max_intensity-min_intensity)
hist_mem = cle.pull(hist)
del im, hist
t = skimage.filters.threshold_multiotsu(image=scan, hist=hist_mem, classes=3)
t = t+min_intensity
mask_hist = np.zeros_like(scan, dtype=np.uint8)
mask_hist[scan <= t[0]] = 1
mask_hist[(scan > t[0]) & (scan < t[1])] = 2
mask_hist[scan >= t[1]] = 3



In [ ]:
hist_skim = skimage.exposure.histogram(scan.astype(np.uint16))

In [ ]:
plt.plot(hist_skim[1], hist_skim[0])
plt.plot(hist_mem)

In [ ]:
t = skimage.filters.threshold_multiotsu(image=scan.astype(np.uint16), classes=3)
mask = np.zeros_like(scan, dtype=np.uint8)
mask[scan <= t[0]] = 1
mask[(scan > t[0]) & (scan < t[1])] = 2
mask[scan >= t[1]] = 3

In [ ]:
viewer = napari.Viewer()
viewer.add_image(scan)
viewer.add_labels(mask)
viewer.add_labels(mask_hist)

In [ ]:
d = cle.list_operations()
print(d)

In [ ]:
cle.mask

In [ ]:
seg_particle = _segment_particles(im)

In [ ]:
plt.imshow(seg_particle)

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
im = clahe.apply(im)
img = cle.gaussian_blur(im, sigma_x=2.2, sigma_y=2.2, sigma_z=2.2)

In [ ]:
non_particle_image = np.ma.masked_array(img, mask=seg_particle)

In [ ]:
th = skimage.filters.threshold_otsu(cle.pull(img)[~seg_particle])
th

In [ ]:
ths = skimage.filters.threshold_multiotsu(cle.pull(img), classes=3)
ths

In [ ]:
binary_non_particle = img > th
seg = img > ths[0]

In [ ]:
plt.imshow(img)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(im)
viewer.add_labels(seg)
viewer.add_labels(binary_non_particle)

In [ ]:


air_particle = contact_area(seg, 1, 3)
polymer_particle = contact_area(seg, 2, 3)

In [ ]:
total_particle_area = air_particle + polymer_particle

print("Surface in contact with air: {:.2f} %".format(air_particle / total_particle_area * 100))
print("Surface in contact with polymer: {:.2f} %".format(polymer_particle / total_particle_area * 100))